##### <h2 align="center">Machine Learning</h2> 
<h3 align="center">Travis Millburn<br>Fall 2020</h3> 

<center>
<img src="../images/logo.png" alt="drawing" style="width: 300px;"/>
</center>

<h3 align="center">Week 12: Deep Learning (Continued)</h3> 

### Outline
1. Semester Projects
2. Deep Learning Concepts Continuation
3. Keras Example
4. Lab

### Projects

Requirements: 
1. Must be python, delivered in Jupyter notebook.
2. Must be supervised learning problem.

Rubric:
1. Data selection interesting/challenging?
2. Method interesting/challenging/thorough?
3. Validation thorough, done properly?
4. Report - well-written/understandable, explains pros/cons of method, what happened & why (can use jupyter completely if use markdown & latex).

Note: some related tasks will go into participation or homework grade.

Project Proposal due 11/16.  Next week.

"Birds inspired us to fly, burdock plants inspired Velcro, and nature has inspired countless more inventions. It seems only logical, then, to look at the brain’s architecture for inspiration on how to build an intelligent machine."

Hands-On Machine Learning ...  
Aurélien Géron

### Gradient Descent
Gradient Descent is a generic optimization algorithm capable of finding optimal solutions to a wide range of problems. The general idea of Gradient Descent is to tweak parameters iteratively in order to minimize a cost function.

<center>
<img src="../images/gradient_descent_2.png" alt="drawing" style="width: 900px;"/>
</center>

<center>
<img src="../images/steps_too_small.png" alt="drawing" style="width: 900px;"/>
</center>

<center>
<img src="../images/steps_too_big.png" alt="drawing" style="width: 900px;"/>
</center>

In [1]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"

In [2]:
import pandas
from sklearn.svm import SVC
import sklearn
from sklearn import preprocessing

from sklearn.model_selection import train_test_split

col_names = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

df = pandas.read_csv('adult.csv')
df.columns=col_names
# df['Income'] = df['Income'].apply(lambda x: 0 if x == ' <=50K' else 1)

for column in df.columns:
    if df[column].dtype == type(object):
        le = sklearn.preprocessing.LabelEncoder()
        df[column] = le.fit_transform(df[column])
df.tail()

,Age,WorkClass,fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
32556,22,4,310152,15,10,4,11,1,4,1,0,0,40,39,0
32557,27,4,257302,7,12,2,13,5,4,0,0,0,38,39,0
32558,40,4,154374,11,9,2,7,0,4,1,0,0,40,39,1
32559,58,4,151910,11,9,6,1,4,4,0,0,0,40,39,0
32560,22,4,201490,11,9,4,1,3,4,1,0,0,20,39,0


In [ ]:
# X = df.drop(columns='Income')

In [ ]:
# type(X)

In [ ]:
# pandas.DataFrame(preprocessing.StandardScaler().fit_transform(X))

In [3]:
X = df.drop(columns='Income')
X = pandas.DataFrame(preprocessing.StandardScaler().fit_transform(X), columns=X.columns)
y = df['Income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=555, stratify=y)

In [4]:
import tensorflow as tf

In [5]:
import numpy as np
nn_X_train = np.reshape(X_train.values, (X_train.shape[0], X_train.shape[1], 1))
nn_X_test = np.reshape(X_test.values, (X_test.shape[0], X_test.shape[1], 1))

In [9]:
# create model
mdl_k = tf.keras.Sequential([tf.keras.layers.Input(shape=(nn_X_train.shape[1], nn_X_train.shape[2]))])
mdl_k.add(tf.keras.layers.Dense(12, input_dim=3, activation='relu', kernel_initializer="he_normal"))
# mdl_k.add(tf.keras.layers.Dense(8, activation='relu'))

mdl_k.add(tf.keras.layers.LSTM(64))
#mdl_k.add(tf.keras.layers.LeakyReLU(alpha=0.2))
mdl_k.add(tf.keras.layers.Dense(1, activation='relu'))

InvalidArgumentError: assertion failed: [0] [Op:Assert] name: EagerVariableNameReuse

In [ ]:
# Compile model
mdl_k.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])

In [ ]:
mdl_k.summary()

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((nn_X_test, y_test)).batch(batch_size=128)

In [ ]:
history = mdl_k.fit(
    nn_X_train,
    y_train,
    epochs=50,
    batch_size=128,
    # validation_data=(nn_X_test, y_test),
    validation_data=val_dataset,
    #validation_split=0.1,
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", verbose=2)
#     ],
    use_multiprocessing=True
)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# We see a bit of overfitting above.